In [2]:
import monai
import torch
import os
import nibabel as nib
import glob
import monai.data as data
from monai.transforms import LoadImaged
import monai.transforms as transforms
from monai.data import DataLoader, CacheDataset

2023-06-01 22:53:19.950406: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 22:53:22.196735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 22:53:26.240445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data Loading

In [3]:
image_dir = "/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr"
label_dir = "/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr"


In [4]:
files_pattern = "*.nii.gz"


In [5]:
#Get a list of image file paths
image_files = sorted(glob.glob(f"{image_dir}/{files_pattern}"))

#Get a list of label file paths
label_files = sorted(glob.glob(f"{label_dir}/{files_pattern}"))

In [6]:
image_files

['/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_00.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_01.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_02.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_04.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_06.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_07.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_10.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_13.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_14.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/imagesTr/prostate_16.nii.gz',
 '/home/richard/Proj

In [7]:
label_files

['/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_00.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_01.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_02.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_04.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_06.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_07.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_10.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_13.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_14.nii.gz',
 '/home/richard/Projects/Datasets/ProstrateDatasets/Task05_Prostate/labelsTr/prostate_16.nii.gz',
 '/home/richard/Proj

In [8]:
images = []
labels = []

for imagename in image_files:
    image_input = nib.load(imagename)
    image = image_input.get_fdata()
    images.append(image)
    
for labelname in label_files:
    label_input = nib.load(labelname)
    label =  label_input.get_fdata()
    labels.append(label)

In [9]:
labels

[array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [10]:
images

[array([[[[6.200e+01, 0.000e+00],
          [5.000e+01, 0.000e+00],
          [4.100e+02, 0.000e+00],
          ...,
          [1.048e+03, 0.000e+00],
          [1.021e+03, 0.000e+00],
          [1.040e+03, 0.000e+00]],
 
         [[6.300e+01, 0.000e+00],
          [1.890e+02, 0.000e+00],
          [5.470e+02, 0.000e+00],
          ...,
          [1.074e+03, 0.000e+00],
          [1.042e+03, 0.000e+00],
          [7.770e+02, 0.000e+00]],
 
         [[9.400e+01, 0.000e+00],
          [2.840e+02, 0.000e+00],
          [6.800e+02, 0.000e+00],
          ...,
          [1.057e+03, 0.000e+00],
          [1.011e+03, 0.000e+00],
          [7.160e+02, 0.000e+00]],
 
         ...,
 
         [[3.350e+02, 0.000e+00],
          [6.700e+01, 0.000e+00],
          [7.000e+00, 0.000e+00],
          ...,
          [6.000e+00, 0.000e+00],
          [8.000e+00, 0.000e+00],
          [8.000e+00, 0.000e+00]],
 
         [[1.690e+02, 0.000e+00],
          [1.900e+01, 0.000e+00],
          [2.400e+01, 0.000e

In [11]:
print(dir(transforms))

['Activations', 'ActivationsD', 'ActivationsDict', 'Activationsd', 'AddChannel', 'AddChannelD', 'AddChannelDict', 'AddChanneld', 'AddCoordinateChannels', 'AddCoordinateChannelsD', 'AddCoordinateChannelsDict', 'AddCoordinateChannelsd', 'AddExtremePointsChannel', 'AddExtremePointsChannelD', 'AddExtremePointsChannelDict', 'AddExtremePointsChanneld', 'AdjustContrast', 'AdjustContrastD', 'AdjustContrastDict', 'AdjustContrastd', 'Affine', 'AffineD', 'AffineDict', 'AffineGrid', 'Affined', 'AsChannelFirst', 'AsChannelFirstD', 'AsChannelFirstDict', 'AsChannelFirstd', 'AsChannelLast', 'AsChannelLastD', 'AsChannelLastDict', 'AsChannelLastd', 'AsDiscrete', 'AsDiscreteD', 'AsDiscreteDict', 'AsDiscreted', 'BatchInverseTransform', 'BorderPad', 'BorderPadD', 'BorderPadDict', 'BorderPadd', 'BoundingRect', 'BoundingRectD', 'BoundingRectDict', 'BoundingRectd', 'CastToType', 'CastToTypeD', 'CastToTypeDict', 'CastToTyped', 'CenterScaleCrop', 'CenterScaleCropD', 'CenterScaleCropDict', 'CenterScaleCropd', 'C

# Image Preprocessing, Transformation and Augmentation

In [12]:


#converting Images from numpy arrays to tensor
tensor_images = [torch.from_numpy(image) for image in images]

#Converting labels from numpy arrays to tensor
tensor_labels = [torch.from_numpy(label) for label in labels]



In [13]:
tensor_labels

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [14]:
#Determining incase the files have different slices
import numpy as np
a = tensor_images.pop()
b = np.random.choice(tensor_images)

c = tensor_labels.pop()
d = np.random.choice(tensor_labels)


/tmp/ipykernel_4933/1763578967.py:4: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  b = np.random.choice(tensor_images)
/tmp/ipykernel_4933/1763578967.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  b = np.random.choice(tensor_images)
/tmp/ipykernel_4933/1763578967.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the cor

In [15]:
a.shape

torch.Size([320, 320, 20, 2])

In [16]:
b.shape

torch.Size([320, 320, 22, 2])

In [17]:
c.shape

torch.Size([320, 320, 20])

In [18]:
d.shape

torch.Size([320, 320, 20])

In [19]:
#Image Intensity Normalization-NormalizedIntensity

import monai.transforms

normalized_image_tensors = []
intensity_norm = transforms.NormalizeIntensity()
for img_tensor in tensor_images:
    normtensor = intensity_norm(img_tensor)
    normalized_image_tensors.append(normtensor)

In [20]:
normalized_image_tensors

[tensor([[[[-0.6913, -0.8177],
           [-0.7158, -0.8177],
           [ 0.0180, -0.8177],
           ...,
           [ 1.3185, -0.8177],
           [ 1.2635, -0.8177],
           [ 1.3022, -0.8177]],
 
          [[-0.6893, -0.8177],
           [-0.4325, -0.8177],
           [ 0.2973, -0.8177],
           ...,
           [ 1.3715, -0.8177],
           [ 1.3063, -0.8177],
           [ 0.7661, -0.8177]],
 
          [[-0.6261, -0.8177],
           [-0.2388, -0.8177],
           [ 0.5684, -0.8177],
           ...,
           [ 1.3369, -0.8177],
           [ 1.2431, -0.8177],
           [ 0.6418, -0.8177]],
 
          ...,
 
          [[-0.1349, -0.8177],
           [-0.6812, -0.8177],
           [-0.8035, -0.8177],
           ...,
           [-0.8055, -0.8177],
           [-0.8014, -0.8177],
           [-0.8014, -0.8177]],
 
          [[-0.4732, -0.8177],
           [-0.7790, -0.8177],
           [-0.7688, -0.8177],
           ...,
           [-0.7994, -0.8177],
           [-0.8014, -0

In [21]:
#Image Intensity Normalizatio-ScaledIntensity
from monai.transforms import ScaleIntensity
normalized_tensors = []
normaliser = ScaleIntensity(minv = 0.0, maxv = 1.0)
for img_tensor in tensor_images:
    norm_tensor = normaliser(img_tensor)
    normalized_tensors.append(norm_tensor)


In [22]:
normalized_tensors

[tensor([[[[1.7132e-02, 0.0000e+00],
           [1.3816e-02, 0.0000e+00],
           [1.1329e-01, 0.0000e+00],
           ...,
           [2.8958e-01, 0.0000e+00],
           [2.8212e-01, 0.0000e+00],
           [2.8737e-01, 0.0000e+00]],
 
          [[1.7408e-02, 0.0000e+00],
           [5.2224e-02, 0.0000e+00],
           [1.5115e-01, 0.0000e+00],
           ...,
           [2.9677e-01, 0.0000e+00],
           [2.8792e-01, 0.0000e+00],
           [2.1470e-01, 0.0000e+00]],
 
          [[2.5974e-02, 0.0000e+00],
           [7.8475e-02, 0.0000e+00],
           [1.8790e-01, 0.0000e+00],
           ...,
           [2.9207e-01, 0.0000e+00],
           [2.7936e-01, 0.0000e+00],
           [1.9784e-01, 0.0000e+00]],
 
          ...,
 
          [[9.2567e-02, 0.0000e+00],
           [1.8513e-02, 0.0000e+00],
           [1.9342e-03, 0.0000e+00],
           ...,
           [1.6579e-03, 0.0000e+00],
           [2.2106e-03, 0.0000e+00],
           [2.2106e-03, 0.0000e+00]],
 
          [[4.6698e

In [23]:
#Removing spatially varying intensities
from monai.transforms import GaussianSharpen
sharpened_tensors = []
sh = GaussianSharpen()
for im in normalized_tensors:
    norm_tensor = sh(im)
    sharpened_tensors.append(norm_tensor)

In [24]:
sharpened_tensors


[tensor([[[[ 1.3484e-01,  1.2402e-01],
           [ 1.6353e-01,  1.4907e-01],
           [ 2.0681e-01,  1.8834e-01],
           ...,
           [ 2.7537e-01,  2.5162e-01],
           [ 2.3803e-01,  2.1796e-01],
           [ 2.1362e-01,  1.9738e-01]],
 
          [[ 1.5277e-01,  1.3904e-01],
           [ 1.7342e-01,  1.5536e-01],
           [ 2.1448e-01,  1.9171e-01],
           ...,
           [ 2.6807e-01,  2.4051e-01],
           [ 2.3269e-01,  2.0954e-01],
           [ 2.1869e-01,  1.9999e-01]],
 
          [[ 1.8460e-01,  1.6774e-01],
           [ 2.0360e-01,  1.8176e-01],
           [ 2.4718e-01,  2.2001e-01],
           ...,
           [ 2.8518e-01,  2.5455e-01],
           [ 2.4590e-01,  2.2036e-01],
           [ 2.3206e-01,  2.1158e-01]],
 
          ...,
 
          [[ 4.4690e-02,  4.0953e-02],
           [ 3.6408e-02,  3.2666e-02],
           [ 3.1456e-02,  2.7973e-02],
           ...,
           [ 2.5701e-03,  2.2789e-03],
           [ 2.2975e-03,  2.0508e-03],
           [ 

In [25]:
#Image filtering and smoothing
from monai.transforms import GaussianSmooth
smoothed_image = []
smoothing  = GaussianSmooth()
for img in sharpened_tensors:
    smoothed = smoothing(img)
    smoothed_image.append(smoothed)

In [26]:
smoothed_image

[tensor([[[[ 4.6652e-02,  4.5659e-02],
           [ 7.0792e-02,  6.9225e-02],
           [ 8.9312e-02,  8.7291e-02],
           ...,
           [ 1.1356e-01,  1.1108e-01],
           [ 9.6016e-02,  9.3992e-02],
           [ 6.6386e-02,  6.5049e-02]],
 
          [[ 6.8240e-02,  6.6717e-02],
           [ 1.0219e-01,  9.9801e-02],
           [ 1.2730e-01,  1.2424e-01],
           ...,
           [ 1.5479e-01,  1.5119e-01],
           [ 1.3098e-01,  1.2805e-01],
           [ 9.0896e-02,  8.8966e-02]],
 
          [[ 8.2744e-02,  8.0840e-02],
           [ 1.2211e-01,  1.1915e-01],
           [ 1.4975e-01,  1.4599e-01],
           ...,
           [ 1.6976e-01,  1.6560e-01],
           [ 1.4320e-01,  1.3984e-01],
           [ 9.9420e-02,  9.7212e-02]],
 
          ...,
 
          [[ 1.7391e-02,  1.7020e-02],
           [ 2.1196e-02,  2.0707e-02],
           [ 1.9435e-02,  1.8950e-02],
           ...,
           [ 1.5648e-03,  1.5236e-03],
           [ 1.3608e-03,  1.3273e-03],
           [ 

# Data Augmentation

In [27]:
import monai.transforms as transforms

In [28]:
Aug_trans = transforms.Compose([
    transforms.RandRotate(15),#Rotates data at -15 degrees and 15 degrees
    transforms.RandFlip(prob = 1,spatial_axis = [0,1]),#Applies both RandVerticalFlip() and RandHorizontalFlip()
    #transforms.RandSpatialCrop((32,32,2)),
    transforms.RandZoom(min_zoom = 0.7, max_zoom = 1.5)#Shrinks to -0.7 and zooms to 1.5 randomly
])

trans_image_tensors = []
for ims in sharpened_tensors:
    transedIm = Aug_trans(ims)
    trans_image_tensors.append(transedIm)

In [29]:
trans_image_tensors

[tensor([[[[ 2.0026e-02,  2.1219e-02],
           [ 1.7097e-02,  1.7921e-02],
           [ 1.4562e-02,  1.5484e-02],
           ...,
           [ 1.8990e-03,  1.9698e-03],
           [ 1.6861e-03,  1.7593e-03],
           [ 1.5599e-03,  1.5970e-03]],
 
          [[ 2.6316e-02,  2.8303e-02],
           [ 2.1765e-02,  2.2918e-02],
           [ 1.8357e-02,  1.9541e-02],
           ...,
           [ 2.1163e-03,  2.1921e-03],
           [ 1.9015e-03,  1.9748e-03],
           [ 1.8364e-03,  1.8482e-03]],
 
          [[ 3.6480e-02,  3.9166e-02],
           [ 3.0387e-02,  3.1907e-02],
           [ 2.5847e-02,  2.7442e-02],
           ...,
           [ 2.6014e-03,  2.6955e-03],
           [ 2.3416e-03,  2.4299e-03],
           [ 2.2792e-03,  2.2868e-03]],
 
          ...,
 
          [[ 1.7531e-01,  1.7077e-01],
           [ 2.0034e-01,  1.8795e-01],
           [ 2.3819e-01,  2.2455e-01],
           ...,
           [ 2.8957e-01,  3.0275e-01],
           [ 2.5225e-01,  2.6498e-01],
           [ 

# Data Splitting

In [30]:
image_dataset = trans_image_tensors

In [31]:
label_dataset = tensor_labels

In [32]:
image_dataset

[tensor([[[[ 2.0026e-02,  2.1219e-02],
           [ 1.7097e-02,  1.7921e-02],
           [ 1.4562e-02,  1.5484e-02],
           ...,
           [ 1.8990e-03,  1.9698e-03],
           [ 1.6861e-03,  1.7593e-03],
           [ 1.5599e-03,  1.5970e-03]],
 
          [[ 2.6316e-02,  2.8303e-02],
           [ 2.1765e-02,  2.2918e-02],
           [ 1.8357e-02,  1.9541e-02],
           ...,
           [ 2.1163e-03,  2.1921e-03],
           [ 1.9015e-03,  1.9748e-03],
           [ 1.8364e-03,  1.8482e-03]],
 
          [[ 3.6480e-02,  3.9166e-02],
           [ 3.0387e-02,  3.1907e-02],
           [ 2.5847e-02,  2.7442e-02],
           ...,
           [ 2.6014e-03,  2.6955e-03],
           [ 2.3416e-03,  2.4299e-03],
           [ 2.2792e-03,  2.2868e-03]],
 
          ...,
 
          [[ 1.7531e-01,  1.7077e-01],
           [ 2.0034e-01,  1.8795e-01],
           [ 2.3819e-01,  2.2455e-01],
           ...,
           [ 2.8957e-01,  3.0275e-01],
           [ 2.5225e-01,  2.6498e-01],
           [ 

In [33]:
label_dataset

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [34]:
#Trials with torct and monai
from monai.data import Dataset, ArrayDataset
from torch.utils.data import TensorDataset




In [35]:
#trials with sklearn
from sklearn.model_selection import train_test_split
train_images,test_images,train_labels,test_labels = train_test_split(image_dataset, label_dataset,test_size = 0.2, shuffle = False, random_state = 0)

In [36]:
len(train_images)

24

In [37]:
len(train_labels)

24

In [38]:
len(test_images)

7

In [39]:
len(test_labels)

7